In [1]:
import numpy as np
from collections import defaultdict

candidates = np.array([['A', "C", "B", "C"],
                   ["C", "B", "A", "A"],
                   ["D", "D", "C", "D"],
                   ["B", "A", "D", "B"]])

values = np.array([7, 5, 5, 3])
sumValues = np.sum(values)

def removeCandidate(candidatesArray, candidate):
    indexes = np.argwhere(candidatesArray==candidate)
    for index in indexes:
        col = index[0]
        row = index[1]
        candidatesArray[col, row] = ""
    return compressArray(candidatesArray)    

def compressArray(array):    
    for col_index in range(array.shape[1]):
        column = array[:, col_index]
        non_empty_values = column[column != ""]
        column[:] = ""

        column[:len(non_empty_values)] = non_empty_values
        
    non_empty_rows = ~np.all(array == "", axis=1)
    array = array[non_empty_rows]

    return array

## Відносна більшість

In [2]:
def relative(canditatesArray, values):
    result = {"A": 0, "B": 0, "C": 0, "D": 0, }

    for index, value in np.ndenumerate(values):
        candidate = canditatesArray[0][index]
        result[candidate] += value
    return result

relativeCandidates = candidates.copy()
topStudents = []

for i in range(np.shape(candidates)[1]):
    relativeArray = relative(relativeCandidates, values)
    top_student = max(relativeArray, key=relativeArray.get)
    
    topStudents = np.append(topStudents, top_student)
    
    relativeCandidates = removeCandidate(relativeCandidates, top_student)
print(topStudents)

['C' 'A' 'B' 'D']


## Абсолютна більшість

In [3]:
candidatesList = np.array(["A", "B", "C", "D"])
absolutelyCandidates = candidates.copy()
topAbsStudents = []

def get_max_value(data):
    d = defaultdict(list)
    for key, value in data.items():
        d[value].append(key)
    return max(d.items())[1]

for i in range(np.shape(candidates)[1]):
    tempCandidates = absolutelyCandidates.copy()

    relativeArray = relative(tempCandidates, values)
    top_student = max(relativeArray, key=relativeArray.get)
    topValue = relativeArray[top_student]
    
    if topValue/sumValues > 0.5:
        topAbsStudents = np.append(topAbsStudents, top_student)
        
        absolutelyCandidates = removeCandidate(absolutelyCandidates, top_student)
    else:

        data = list(relativeArray.values())
        numpyArray = np.array(data)

        top2 = np.argpartition(numpyArray, -2)[-2:]
        topValues = numpyArray[top2]
        
        topKeys = np.array(list(relativeArray.keys()))[np.argsort(numpyArray)[-2:]]
        result_dict = np.array(list(dict(zip(topKeys, topValues)).keys()))

        tempCandidatesArr = candidatesList.copy()
        
        for i in range(len(result_dict)):
            indexes = np.argwhere(tempCandidatesArr==result_dict[i])
            tempCandidatesArr = np.delete(tempCandidatesArr, indexes)    
        
        for i in range(len(tempCandidatesArr)):
            tempCandidates = removeCandidate(tempCandidates, tempCandidatesArr[i])
        
        relativeArray = relative(tempCandidates, values)

        top_student = max(relativeArray, key=relativeArray.get)


        absolutelyCandidates = removeCandidate(absolutelyCandidates, top_student)

print(topAbsStudents)

['C' 'D']


## Борда


In [4]:
def bord(canditatesArray, values):
    result = {"A": 0, "B": 0, "C": 0, "D": 0, }

    for index, value in np.ndenumerate(canditatesArray):
        candidate = canditatesArray[index[0], index[1]]
        
        result[candidate] += values[index[1]]*(3-index[0])
    return result

bordCandidates = candidates.copy()
topBordStudents = []

for i in range(np.shape(candidates)[1]):
    bordArray = bord(bordCandidates, values)
    top_student = max(bordArray, key=bordArray.get)
    
    topBordStudents = np.append(topBordStudents, top_student)
    
    bordCandidates = removeCandidate(bordCandidates, top_student)
print(topBordStudents)

['C' 'A' 'B' 'D']


## Кондорсе

In [5]:
# методом Кондорсе
def condorcet_method(votes):
    candidates = set([vote for sublist in votes for vote in sublist])
    pairwise_votes = {(c1, c2): 0 for c1 in candidates for c2 in candidates if c1 != c2}
    
    for sublist in votes:
        for i in range(len(sublist)):
            for j in range(i + 1, len(sublist)):
                if sublist[i] != sublist[j]:
                    pairwise_votes[(sublist[i], sublist[j])] += 1
    
    for candidate in candidates:
        pairwise_wins = sum([1 for other_candidate in candidates if other_candidate != candidate and pairwise_votes[(candidate, other_candidate)] > pairwise_votes[(other_candidate, candidate)]])
        if pairwise_wins == len(candidates) - 1:
            return candidate
    
    return None

# методом Копленда
def copeland_method(votes):
    candidates = set([vote for sublist in votes for vote in sublist])
    scores = {candidate: 0 for candidate in candidates}
    
    for c1 in candidates:
        for c2 in candidates:
            if c1 != c2:
                for sublist in votes:
                    try:
                        if sublist.index(c1) < sublist.index(c2):
                            scores[c1] += 1
                        elif sublist.index(c2) < sublist.index(c1):
                            scores[c2] += 1
                    except ValueError:
                        pass
    
    winner = max(scores, key=scores.get)
    return winner

# методом Сімпсона
def simpson_method(votes):
    candidates = set([vote for sublist in votes for vote in sublist])
    scores = {candidate: 0 for candidate in candidates}
    
    for i, sublist in enumerate(votes):
        for j, candidate in enumerate(sublist):
            scores[candidate] += (len(sublist) - j)
    
    winner = max(scores, key=scores.get)
    return winner

print("Кондорсе:", condorcet_method(candidates))
print("Копленд:", copeland_method(candidates.tolist()))
print("Сімпсон:", simpson_method(candidates))

Кондорсе: None
Копленд: C
Сімпсон: C
